In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="0"; 

import urllib.request
import pandas as pd
import numpy as np
import time
import ktrain
from ktrain import tabular
from sklearn.model_selection import train_test_split
from tqdm import *
pd.set_option('display.max_columns', None)

from benchmark_circuitsv2 import *
import random
import pandas as pd
from tqdm import *
import pkgutil
import warnings
import exrex
import math
import time
import json
import scipy.stats as stats
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
warnings.filterwarnings('ignore')


def getIntegers(startRange,upperlimit):
    return [x for x in range(startRange,startRange+upperlimit)]

def getallBinaryValues(number_bits):
    max_number = np.power(2,number_bits)-1
    data = []
    for x in range(0,max_number):
        value = bin(x).replace("0b","")
        value = "0"*(number_bits-len(value))+value
        data.append(value)
    return data

def getexpression(regex,upperlimit):
    return [exrex.getone(regex) for x in range(upperlimit)]


def generate_data(Format,startRange,endRange,percentage,regex,circuit):
    data = []
    if Format=="int":
        upperlimit = int(np.ceil((endRange-startRange)*percentage))
        data = getIntegers(startRange,upperlimit)
    if Format=="binary":
        upperlimit = int(np.ceil((endRange-startRange)*percentage))
        data = getallBinaryValues(startRange+upperlimit)
    if Format=="expression":
        upperlimit = int(np.ceil((math.factorial(startRange))*percentage))
        data = getexpression(regex,upperlimit)
    return data,circuit

def read_configuration(filepath):
    with open(filepath,"r") as file:
        content = file.read()
    rules = content.split("-"*20)
    rules_dict = {}
    for rule in rules:
        try:
            parameters = rule.split("\n")
            parameter_dict = {}
            for parameter in parameters:
                if parameter!="":
                    key,value = parameter.split(":")
                    value = value.strip()
                    if value.isdigit():
                        value = int(value)
                    else:
                        try:
                            value = float(value)
                        except:
                            pass
                    parameter_dict[key] = value
            rules_dict[parameter_dict.pop("ID")] = parameter_dict
        except:
            continue
    return rules_dict


# # Select CUTs

# In[2]:


backends = [('FakeAlmaden', 20), ('FakeBoeblingen', 20), ('FakeBrooklyn', 65), ('FakeCairo', 27), ('FakeCambridge', 28), ('FakeCambridgeAlternativeBasis', 28), ('FakeCasablanca', 7), ('FakeGuadalupe', 16), ('FakeHanoi', 27), ('FakeJakarta', 7), ('FakeJohannesburg', 20), ('FakeKolkata', 27), ('FakeLagos', 7), ('FakeManhattan', 65), ('FakeMontreal', 27), ('FakeMumbai', 27), ('FakeNairobi', 7), ('FakeParis', 27), ('FakeRochester', 53), ('FakeSingapore', 20), ('FakeSydney', 27), ('FakeToronto', 27), ('FakeWashington', 127)]
BaselineCircuits,CUTs = train_test_split(get_all_circuits(),train_size=0.4,random_state=13)


# # Get Evaluation data for CUTs

# In[3]:


# rules = read_configuration("Configuration.txt")
# data_circuit_pairs = []
# for baseline_circuit in CUTs:
#     circuit = get_circuit_class_object(baseline_circuit)
#     print("Generating data for CUT circuit: {}, ID:{}".format(baseline_circuit,circuit.key_aurguments["ID"]))
#     rule = rules[circuit.key_aurguments["ID"]]
#     data,circuit = generate_data(Format=rule["FORMAT"],startRange=rule["START"],
#                                  endRange=rule["END"],percentage=rule["PERCENTAGE"],
#                                  regex=rule["REGEX"],circuit=circuit)
#     data_circuit_pairs.append((data,circuit,baseline_circuit))

# data_circuit_pairs
data_circuit_pairs = []
for baseline_circuit in CUTs:
    circuit = get_circuit_class_object(baseline_circuit)
    data_circuit_pairs.append((circuit.get_full_inputs(),circuit,baseline_circuit))

data_circuit_pairs

[([3, 4, 5], <benchmark_circuitsv2.ghz at 0x1c3384dd250>, 'ghz'),
 (['0000',
   '0001',
   '0010',
   '0011',
   '0100',
   '0101',
   '0110',
   '0111',
   '1000',
   '1001',
   '1010',
   '1011',
   '1100',
   '1101',
   '1110',
   '1111'],
  'phase'),
 ([(1, 1), (1, 2), (1, 3), (2, 1), (2, 2), (2, 3), (3, 1), (3, 2), (3, 3)],
  'addition'),
 (['000', '001', '010', '011', '100', '101', '110', '111'],
  'simon'),
 (['00000',
   '00001',
   '00010',
   '00011',
   '00100',
   '00101',
   '00110',
   '00111',
   '01000',
   '01001',
   '01010',
   '01011',
   '01100',
   '01101',
   '01110',
   '01111',
   '10000',
   '10001',
   '10010',
   '10011',
   '10100',
   '10101',
   '10110',
   '10111',
   '11000',
   '11001',
   '11010',
   '11011',
   '11100',
   '11101',
   '11110',
   '11111'],
  'qft'),
 ([('000', '000'),
   ('000', '001'),
   ('000', '010'),
   ('000', '011'),
   ('000', '100'),
   ('000', '101'),
   ('000', '110'),
   ('000', '111'),
   ('001', '000'),
   ('001', '001'

In [2]:
# # Execute on Backends

# In[4]:


backend_factory = BackendFactory()
backend = backend_factory.initialize_backend()
backend_executor = {}
for bk, qubit_size in tqdm(backends):
    backend_executor[bk] = backend_factory.initialize_backend(bk)
    
seed = 1977
for inter in tqdm(range(10)):
    seed+=20
    for bk, qubit_size in backends:

        #print("Generating Data For {} Backend".format(bk))
        #print("------------------------------------------")

        for data,circuit,name in data_circuit_pairs:
            noise_data = []
            ideal_data = []
            #print("Executing CUT circuit ID: {}".format(circuit.key_aurguments["ID"]))

            if circuit.key_aurguments["input_type"]==1:
                ideal = circuit.get_result(backend,data,seed=seed)
                Noise = circuit.get_result(backend_executor[bk],data,seed=seed)
                for inp in data:
                    noise_data.append({"{}".format(inp):Noise[inp]})
                    ideal_data.append({"{}".format(inp):ideal[inp]})

            elif circuit.key_aurguments["input_type"]==2:
                ideal = circuit.get_result(backend,data,seed=seed)
                Noise = circuit.get_result(backend_executor[bk],data,seed=seed)
                for inp in data:
                    noise_data.append({"{}:{}".format(inp[0],inp[1]):Noise[inp]})
                    ideal_data.append({"{}:{}".format(inp[0],inp[1]):ideal[inp]})

        # #-=-=-=-=-=-=-==-=-=-=-=-=-=-==-Saving json-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
            data = {"ideal":ideal_data,"noise":noise_data}
            file = open("density/_{}_{}_{}.json".format(inter,bk,name),"w")
            json.dump(data,file)
            file.close()

100%|███████████████████████████████████████████████████████████████████████████████| 10/10 [2:44:46<00:00, 988.62s/it]
